# **PLQY_individual**

_by Felix Eickemeyer_

PLQY, this notebook is based on "PLQY 1.3.4.ipynb".
Calibration and correction has to be done before.
__

_Version 10.11.2021 in Python 3_

_Change log:_  
0.1.0: My package used  
1.0.0: First version bug free and usable for thot desktop  
1.1.0: All graphs saved as assets  
1.2.0: Experminetal parameters as own class (exp_param)   

In [55]:
import os
from os import getcwd, listdir
import thot
from thot import ThotProject
import importlib
from importlib import reload

from FTE_analysis_libraries import PLQY as lqy
from FTE_analysis_libraries import Spectrum as spc
from FTE_analysis_libraries.General import f1240, Vsq, V_loss, QFLS

reload(spc)
reload(lqy)
pass

In [56]:
# Initializes Thot project
db = ThotProject( dev_root = '../double_perovskite_temperature' )

In [57]:
root = db.find_container( { '_id': db.root } )

In [58]:
#Perovskite
if 'sample_type' in root.metadata:
    which_sample = root.metadata[ 'sample_type' ]

else:
    pass
    #which_sample = 'FAPbI3'

#DSC
#which_sample = 'Yameng DSC'
#which_sample = 'dye on TiO2'
#which_sample = 'dye on Al2O3'
#which_sample = 'Coumarin 153'
#which_sample = 'MS5'
#which_sample = 'XY1b'

param = lqy.exp_param(which_sample = which_sample, excitation_laser = None, PL_left = None, PL_right = None, PL_peak = None, corr_offs_left = 40, corr_offs_right = 50, PL_peak_auto = False, eval_Pb = False)

In [59]:
samples = db.find_assets({'type' : 'calibrated PL spectrum'})
names = list({sample.metadata['name'] for sample in samples})
names.remove('no sample')
names

['12', '14', '6', '7', '13', '3', '10', '1']

In [60]:
La = lqy.find({'metadata.name' : 'no sample', 'metadata.em_filter' : param.laser_marker}, samples, show_details = True)
Pa = lqy.find({'metadata.name' : 'no sample', 'metadata.em_filter' : param.PL_marker}, samples, show_details = True)

no sample--ip_laser--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_420BPF.csv
no sample--ip_laser--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_450LPF.csv


In [61]:
for idx in range(len(names)):
    show_details = True
    sample_name = names[idx]
    group = thot.filter({'metadata.name' : sample_name}, samples)
    Lb = lqy.find({'metadata.em_filter' : param.laser_marker, 'metadata.inboob' : 'outofbeam'}, group, show_details = show_details)
    Lc = lqy.find({'metadata.em_filter' : param.laser_marker, 'metadata.inboob' : 'inbeam'}, group, show_details = show_details)
    Pb = lqy.find({'metadata.em_filter' : param.PL_marker, 'metadata.inboob' : 'outofbeam'}, group, show_details = show_details)
    Pc = lqy.find({'metadata.em_filter' : param.PL_marker, 'metadata.inboob' : 'inbeam'}, group, show_details = show_details)
    fs = lqy.find({'metadata.em_filter' : param.PL_marker, 'metadata.fsip' : 'fs'}, group, show_details = show_details)

    show_details = True

    sPL = lqy.PLQY_dataset(db, La, Lb, Lc, Pa, Pb, Pc, fs, sample_name, param)
    #sPL.fs.plot(yscale = 'linear', title = sPL.fs_asset.metadata['orig_fn'])
    #sPL.P.plot()

    sPL.find_PL_peak()
    sPL.inb_adjust(adj_factor = None, show_adjust_factor = False, show = show_details)
    sPL.calc_abs(what = 'inb', show_details = show_details)

    #sPL.oob_adjust(adj_factor = None, show_adjust_factor = True, show = True)
    #sPL.calc_abs(what = 'oob', show_details = show_details)

    sPL.calc_PLQY(show = show_details, show_lum = 'linear')

    sPL.abs_pf_spec(nsuns = 1)

    sPL.save_asset()

12--ip_outofbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_420BPF.csv
12--ip_inbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_420BPF.csv
12--ip_outofbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_450LPF.csv
12--ip_inbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_450LPF.csv
12--fs--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_450LPF.csv
La = 5.25e+15 1/(s m2)
Lb = 6.70e+15 1/(s m2)
Lc = 1.53e+14 1/(s m2)
Pa = 0.00e+00 1/(s m2)
Pb = 0.00e+00 1/(s m2)
Pc = 1.45e+14 1/(s m2)
A = 1 - Lc/Lb = 97.7 %
PLQY = (Pc - (1 - A) * Pb) / (La * A) = 2.83e-02
File saved: /home/brian/Dropbox/Documents/School/EPFL/research/plqy_analysis/double_perovskite_temperature/12_absolute PL spectrum/12_absolute PL spectrum.csv
14--ip_outofbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_420BPF.csv
14--ip_inbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_420BPF.csv
14--ip

/home/brian/Dropbox/Documents/School/EPFL/research/plqy_analysis/analysis/FTE-analysis-libraries-main/FTE_analysis_libraries/General.py:186: RuntimeWarning: invalid value encountered in log
  return - k * T_RT / q * np.log(PLQY)


File saved: /home/brian/Dropbox/Documents/School/EPFL/research/plqy_analysis/double_perovskite_temperature/14_absolute PL spectrum/14_absolute PL spectrum.csv
6--ip_outofbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_420BPF.csv
6--ip_inbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_420BPF.csv
6--ip_outofbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_450LPF.csv
6--ip_inbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_450LPF.csv
6--fs--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_450LPF.csv
La = 5.25e+15 1/(s m2)
Lb = 9.08e+15 1/(s m2)
Lc = 9.51e+14 1/(s m2)
Pa = 0.00e+00 1/(s m2)
Pb = 0.00e+00 1/(s m2)
Pc = -1.69e+13 1/(s m2)
A = 1 - Lc/Lb = 89.5 %
PLQY = (Pc - (1 - A) * Pb) / (La * A) = -3.61e-03


/home/brian/Dropbox/Documents/School/EPFL/research/plqy_analysis/analysis/FTE-analysis-libraries-main/FTE_analysis_libraries/General.py:186: RuntimeWarning: invalid value encountered in log
  return - k * T_RT / q * np.log(PLQY)


File saved: /home/brian/Dropbox/Documents/School/EPFL/research/plqy_analysis/double_perovskite_temperature/6_absolute PL spectrum/6_absolute PL spectrum.csv
7--ip_outofbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_420BPF.csv
7--ip_inbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_420BPF.csv
7--ip_outofbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_450LPF.csv
7--ip_inbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_450LPF.csv
7--fs--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_450LPF.csv
La = 5.25e+15 1/(s m2)
Lb = 8.94e+15 1/(s m2)
Lc = 3.63e+15 1/(s m2)
Pa = 0.00e+00 1/(s m2)
Pb = 0.00e+00 1/(s m2)
Pc = 3.17e+13 1/(s m2)
A = 1 - Lc/Lb = 59.4 %
PLQY = (Pc - (1 - A) * Pb) / (La * A) = 1.02e-02
File saved: /home/brian/Dropbox/Documents/School/EPFL/research/plqy_analysis/double_perovskite_temperature/7_absolute PL spectrum/7_absolute PL spectrum.csv
13--ip_outofbeam--laser_419nm_10mW_OD

/home/brian/Dropbox/Documents/School/EPFL/research/plqy_analysis/analysis/FTE-analysis-libraries-main/FTE_analysis_libraries/General.py:186: RuntimeWarning: invalid value encountered in log
  return - k * T_RT / q * np.log(PLQY)


File saved: /home/brian/Dropbox/Documents/School/EPFL/research/plqy_analysis/double_perovskite_temperature/13_absolute PL spectrum/13_absolute PL spectrum.csv
3--ip_outofbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_420BPF.csv
3--ip_inbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_420BPF.csv
3--ip_outofbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_450LPF.csv
3--ip_inbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_450LPF.csv
3--fs--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_450LPF.csv
La = 5.25e+15 1/(s m2)
Lb = 5.34e+15 1/(s m2)
Lc = 2.36e+15 1/(s m2)
Pa = 0.00e+00 1/(s m2)
Pb = 0.00e+00 1/(s m2)
Pc = 6.02e+13 1/(s m2)
A = 1 - Lc/Lb = 55.8 %
PLQY = (Pc - (1 - A) * Pb) / (La * A) = 2.06e-02
File saved: /home/brian/Dropbox/Documents/School/EPFL/research/plqy_analysis/double_perovskite_temperature/3_absolute PL spectrum/3_absolute PL spectrum.csv
10--ip_outofbeam--laser_419nm_10mW_

/home/brian/Dropbox/Documents/School/EPFL/research/plqy_analysis/analysis/FTE-analysis-libraries-main/FTE_analysis_libraries/General.py:186: RuntimeWarning: invalid value encountered in log
  return - k * T_RT / q * np.log(PLQY)


File saved: /home/brian/Dropbox/Documents/School/EPFL/research/plqy_analysis/double_perovskite_temperature/10_absolute PL spectrum/10_absolute PL spectrum.csv
1--ip_outofbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_420BPF.csv
1--ip_inbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_420BPF.csv
1--ip_outofbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_450LPF.csv
1--ip_inbeam--laser_419nm_10mW_OD1--Andor_0.04999s_200acc_300lmm_center620nm_450LPF.csv
Error: {'metadata.em_filter': '450LPF', 'metadata.fsip': 'fs'} found more than one instance!


AttributeError: 'NoneType' object has no attribute 'file'